# 5.1 인스타그램 크롤링 

### 5.1.2 인스타그램 접속 후 로그인하기

In [1]:
# 예제 5-1 chromedriver 열기
import chromedriver_autoinstaller
from selenium import webdriver
from bs4 import BeautifulSoup

chromedriver_autoinstaller.install()
driver = webdriver.Chrome()


#### 안내사항
로그인을 해야만 게시물을 볼 수 있도록 인스타그램 운영 정책이 변경되었습니다(2020.04.27)  
인스타그램 로그인을 하는 코드를 추가하였습니다.   
직접 브라우저에 계정 정보를 입력해도 되고, 코드를 이용해도 됩니다.   
다만,  본인의 계정/비번 정보가 외부에 노출되지 않도록 주의하시기 바랍니다. 

※ 주의) `인스타 로그인`을 위해 계정 `아이디` / `비번` 입력이 필요합니다. 

In [2]:
# 예제 5-2 인스타그램 접속하기
import time

# 인스타그램 접속하기
driver.get('https://www.instargram.com')
time.sleep(2)

In [3]:
button = driver.find_element_by_css_selector('#details-button')
button.click()
link = driver.find_element_by_css_selector('#proceed-link')
link.click()


/tmp/ipykernel_9393/2949803461.py:1: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  button = driver.find_element_by_css_selector('#details-button')
/tmp/ipykernel_9393/2949803461.py:3: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  link = driver.find_element_by_css_selector('#proceed-link')


In [4]:
# driver.get('#') 작동안함.

---

#### ※ 로그인 페이지로 접속이 되지 않고 오류 화면이 보일 경우 아래 링크를 참고하세요
https://www.notion.so/playwithdata/c02e510507504b42ae6073c1fbb46f29

---

In [5]:
# 예제 5-3 인스타계정으로 로그인하기
with open('private', 'r') as f:
    email = f.readline().strip()
    password = f.readline().strip()
   ### 계정 정보 수정 필요
input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
input_id.clear()
time.sleep(0.3)
input_id.send_keys(email)

 ### 비번 정보 수정 필요
input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]
input_pw.clear()
time.sleep(0.3)
input_pw.send_keys(password)
input_pw.submit()
time.sleep(3)

/tmp/ipykernel_9393/316514582.py:6: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
/tmp/ipykernel_9393/316514582.py:12: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]


In [7]:
# 로그인 정보 저장하겠냐는 메세지 
button = driver.find_elements_by_css_selector('button.sqdOP.yWX7d.y3zKF')[0]
button.click()

/tmp/ipykernel_9393/2168658101.py:2: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  button = driver.find_elements_by_css_selector('button.sqdOP.yWX7d.y3zKF')[0]


In [8]:
# 알림을 키라는 메세지에 싫다는 의사
button = driver.find_elements_by_css_selector('.aOOlW.HoLwm')[0]
button.click()


/tmp/ipykernel_9393/3549363966.py:5: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  button = driver.find_elements_by_css_selector('.aOOlW.HoLwm')[0]


위 코드 실행후 인스타그램 로그인이 완료되어야 합니다.  

추가로, 2단계 인증이 필요하다고 나올 경우 인증까지 완료해주세요. 

인스타계정이 아닌 페이스북계정으로 로그인이 필요하거나  코드에 아이디/비번 입력하는 것이 부담되는 분은

수기로 직접 입력해서 로그인까지 완료하시면 됩니다. 



### 5.1.3 인스타그램 검색결과 URL 만들어서 접속하기

In [9]:
# 예제 5-4 검색결과 URL 만드는 함수
def insta_searching(word):
    
    url = 'https://www.instagram.com/explore/tags/' + word
    return url    

In [10]:
# 예제 5-5 검색결과 페이지 접속하기
word = "제주도맛집"
url = insta_searching(word)
driver.get(url)

### 5.1.4 첫 번째 게시글 열기 

In [11]:
# 예제 5-6 HTML에서 첫번째 게시글 찾아 클릭하기
def select_first(driver):
    first = driver.find_element_by_css_selector("div._9AhH0")
    first.click()
    time.sleep(1)
    
select_first(driver)

/tmp/ipykernel_9393/3062274932.py:3: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  first = driver.find_element_by_css_selector("div._9AhH0")


### 5.1.5 게시글 정보 가져오기 

In [34]:
# 예제 5-7 게시글 정보 가져오기
import re
from bs4 import BeautifulSoup
import unicodedata

def get_content(driver):
    # ① 현재 페이지 html 정보 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    # ② 본문 내용 가져오기
    try:
        content = soup.select('div.C4VMK')[0].text
        # content = unicodedata.normalize('NFC', content) 
    except:
        content = ' '
    # ③ 본문 내용에서 해시태그 가져오기(정규식 활용)
    tags = re.findall(r'#[^\s#,\\]+', content)  
    # ④ 작성일자 정보 가져오기
    date = soup.select('time._1o9PC')[0]['datetime'][:10]
    # ⑤ 좋아요 수 가져오기
    try: 
        # _7UhW9   xLCgt      MMzan   KV-D4           uL8Hv        T0kll # 이름
        # _7UhW9   xLCgt        qyrsm KV-D4               fDxYl    T0kll # 수
        # _7UhW9   xLCgt        qyrsm KV-D4           uL8Hv        T0kll # views 
        like = soup.select('div._7UhW9.xLCgt.qyrsm.fDxYl')[0].text#[0].text[4:-1]   
    except:
        like = 0
    # ⑥ 위치정보 가져오기
    try: 
        place = soup.select('div.M30cS')[0].text
        place = unicodedata.normalize('NFC', place)
    except:
        place = ''
    # ⑦ 수집한 정보 저장하기
    data = [content, date, like, place, tags]
    return data

get_content(driver)

['redrabbit_973• 영국찻집워낙 유명한 곳이라 언제나 뒤로 미뤘는데, 급 들렸다 횡재💚 너무... 너무 아름다와.... 💚 .....#제주 #제주도 #제주맛집 #제주도맛집 #제주카페 #제주카페추천 #애월카페 #애월카페추천 #제주여행 #여행 #여행스타그램 #여행에미치다31m',
 '2022-05-10',
 '58 likes',
 '영국찻집',
 ['#제주',
  '#제주도',
  '#제주맛집',
  '#제주도맛집',
  '#제주카페',
  '#제주카페추천',
  '#애월카페',
  '#애월카페추천',
  '#제주여행',
  '#여행',
  '#여행스타그램',
  '#여행에미치다31m']]

### 5.1.6 다음 게시글 열기

In [13]:
# 예제 5-8 다음 게시글 열기
def move_next(driver):

    right = driver.find_element_by_css_selector('.l8mY4 button')
    # print(right)
    right.send_keys('\n')
    time.sleep(1)

move_next(driver)

/tmp/ipykernel_9393/3818341435.py:4: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  right = driver.find_element_by_css_selector('.l8mY4 button')


In [37]:
move_next(driver)

/tmp/ipykernel_9393/3818341435.py:4: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  right = driver.find_element_by_css_selector('.l8mY4 button')


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".l8mY4 button"}
  (Session info: chrome=100.0.4896.60)
Stacktrace:
#0 0x55985a8b9013 <unknown>
#1 0x55985a61ba08 <unknown>
#2 0x55985a651fb6 <unknown>
#3 0x55985a652171 <unknown>
#4 0x55985a684dd4 <unknown>
#5 0x55985a66f95d <unknown>
#6 0x55985a682b94 <unknown>
#7 0x55985a66f823 <unknown>
#8 0x55985a64508a <unknown>
#9 0x55985a6461e5 <unknown>
#10 0x55985a8e6e5d <unknown>
#11 0x55985a900061 <unknown>
#12 0x55985a8e8bb5 <unknown>
#13 0x55985a900e08 <unknown>
#14 0x55985a8dc2f0 <unknown>
#15 0x55985a91c988 <unknown>
#16 0x55985a91cb08 <unknown>
#17 0x55985a9365dd <unknown>
#18 0x7feff7d62609 <unknown>


### 5.1.7여러 게시글 정보 수집하기

In [1]:
# 예제 5-9 인스타그램 크롤링 ①
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re

# ① 크롬 브라우저 열기
driver = webdriver.Chrome() 

# 추가) 인스타그램 접속후, 로그인 하기  - 
driver.get('https://www.instargram.com')
time.sleep(2)

button = driver.find_element_by_css_selector('#details-button')
button.click()
link = driver.find_element_by_css_selector('#proceed-link')
link.click()
time.sleep(2)
######## 인스타 계정 로그인이 필요합니다 #########
# 아이디 입력
input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
input_id.clear()
time.sleep(0.3)
input_id.send_keys(email)
# 비밀번호 입력
input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]
input_pw.clear()
time.sleep(0.3)
input_pw.send_keys(password)
input_pw.submit()
time.sleep(3)
# 로그인 정보 저장하겠냐는 메세지 
button = driver.find_elements_by_css_selector('button.sqdOP.yWX7d.y3zKF')[0]
button.click()
time.sleep(2)
# 알림을 키라는 메세지에 싫다는 의사를 내비치자!
button = driver.find_elements_by_css_selector('.aOOlW.HoLwm')[0]
button.click()


WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://chromedriver.chromium.org/home


위 코드 실행후 인스타그램 로그인이 완료되어야 합니다.  

추가로, 2단계 인증이 필요하다고 나올 경우 인증까지 완료해주세요. 

인스타계정이 아닌 페이스북계정으로 로그인이 필요하거나  코드에 아이디/비번 입력하는 것이 부담되는 분은

수기로 직접 입력해서 로그인까지 완료하시면 됩니다. 



In [38]:
# 예제 5-10 인스타그램 크롤링  ②
# ② 인스타그램 검색페이지 URL 만들기
word = "제주도맛집"    #검색어
url = insta_searching(word)

# ③ 검색페이지 접속하기
driver.get(url)
time.sleep(9)

# ④ 첫 번째 게시글 열기
select_first(driver)

# ⑤ 비어있는 변수(results)만들기
results = [ ]


# ⑥→⑦→⑧ 여러 게시물 수집하기
target = 50      # 크롤링할 게시글 수
for i in range(target):
    # 게시글 수집에 오류 발생시(네트워크 문제 등의 이유로)  2초 대기 후, 다음 게시글로 넘어가도록 try, except 구문 활용
    try:
        data = get_content(driver)    # 게시글 정보 가져오기
        results.append(data)
        move_next(driver)
    except:
        time.sleep(2)
        move_next(driver)
    

print(results[:2])

/tmp/ipykernel_9393/3062274932.py:3: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  first = driver.find_element_by_css_selector("div._9AhH0")
/tmp/ipykernel_9393/3818341435.py:4: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  right = driver.find_element_by_css_selector('.l8mY4 button')


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".l8mY4 button"}
  (Session info: chrome=100.0.4896.60)
Stacktrace:
#0 0x55985a8b9013 <unknown>
#1 0x55985a61ba08 <unknown>
#2 0x55985a651fb6 <unknown>
#3 0x55985a652171 <unknown>
#4 0x55985a684dd4 <unknown>
#5 0x55985a66f95d <unknown>
#6 0x55985a682b94 <unknown>
#7 0x55985a66f823 <unknown>
#8 0x55985a64508a <unknown>
#9 0x55985a6461e5 <unknown>
#10 0x55985a8e6e5d <unknown>
#11 0x55985a900061 <unknown>
#12 0x55985a8e8bb5 <unknown>
#13 0x55985a900e08 <unknown>
#14 0x55985a8dc2f0 <unknown>
#15 0x55985a91c988 <unknown>
#16 0x55985a91cb08 <unknown>
#17 0x55985a9365dd <unknown>
#18 0x7feff7d62609 <unknown>


In [39]:
results[:10]

[['qqn667ui8i(광고)지만 제주 공항에서 가깝고 (🚗공항에서 12분)오션뷰& 노을뷰를 보면서 먹을 수 있는 💙삼미횟집💙 싱싱한 회는 물론이고사이드가 다양하게 손님들 모시고 오기 딱이야 🤭⛲삼미 돔모듬-대⛲-회국수 -전복,삼겹살,새우구이-삼계탕- 다양한 해산물 모듬 2판- 해물난장-성게 미역국-생선구이 2종 -매운탕등 ~ 너무 많아서 나열하기 어려운 정도! 🤪가게도 총 3층으로 룸이 나눠져 있으니깐 편하고 프라이빗하게 ~✌🧭삼미횟집제주 제주시 도두항서5길 1064-713-6400#제주도맛집#제주맛집#제주도횟집#제주시횟집#제주공항근처횟집#제주시내횟집#용두암근처맛집#용두암맛집#제주횟집추천#제주도공항근처횟집#도두맛집#이호테우해변맛집#제주공항맛집23h',
  '2022-05-09',
  'others',
  '',
  ['#제주도맛집',
   '#제주맛집',
   '#제주도횟집',
   '#제주시횟집',
   '#제주공항근처횟집',
   '#제주시내횟집',
   '#용두암근처맛집',
   '#용두암맛집',
   '#제주횟집추천',
   '#제주도공항근처횟집',
   '#도두맛집',
   '#이호테우해변맛집',
   '#제주공항맛집23h']],
 ['jejuyogi_.💖제주 "자드부팡"💖요즘 새롭게 생기고 조녜스러운유럽감성으로 유명해지구있는 신상카페얌😎여기서 사진한컷 찍으면...내가 어딧는지전부 모를꺼야!!!ㅠㅠ 한컷 찍으러가쟈!✔주소 : 제주시 조천읍 선흘리 954..#제주맛집 #제주도맛집 #제주공항근처맛집 #제주시맛집 #애월맛집 #중문맛집 #제주공항맛집 #서귀포맛집 #산방산맛집 #신제주맛집 #제주시청맛집 #제주애월맛집 #월정리맛집 #제주여행 #제주핫플19h',
  '2022-05-09',
  '1,103 likes',
  '',
  ['#제주맛집',
   '#제주도맛집',
   '#제주공항근처맛집',
   '#제주시맛집',
   '#애월맛집',
   '#중문맛집',
   '#제주공항맛집',
   '#서귀포맛집',
   '#산방산맛집',
   '#신제주맛집',

In [45]:
# 예제 5-11 크롤링 결과 저장하기

import pandas as pd

results_df = pd.DataFrame(results)
results_df.columns = ['content','date','like','place','tags']
results_df.to_excel('./files/1_crawling_jejudoMatJip.xlsx',index=False)


### 5.1.8 여러 엑셀 파일의 중복을 제거한 후 통합 저장


In [47]:
# 예제 5-12 여러 개의 저장파일 통합하기
jeju_insta_df = pd.DataFrame( [ ] )

folder = './files/'
f_list = ['1_crawling_jejudoMatJip.xlsx', '1_crawling_jejudoGwanGwang.xlsx', '1_crawling_jejuMatJip.xlsx', '1_crawling_jejuYeoHang.xlsx']
for fname in f_list:
    fpath = folder + fname
    temp = pd.read_excel(fpath,)
    jeju_insta_df = jeju_insta_df.append(temp)
jeju_insta_df
jeju_insta_df.columns =['content','data','like','place','tags']


/tmp/ipykernel_9393/1342247775.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  jeju_insta_df = jeju_insta_df.append(temp)
/tmp/ipykernel_9393/1342247775.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  jeju_insta_df = jeju_insta_df.append(temp)
/tmp/ipykernel_9393/1342247775.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  jeju_insta_df = jeju_insta_df.append(temp)
/tmp/ipykernel_9393/1342247775.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  jeju_insta_df = jeju_insta_df.append(temp)


In [48]:
# 예제 5-13 중복 데이터 제거하고 저장하기
jeju_insta_df.drop_duplicates(subset = [ "content"] , inplace = True)
jeju_insta_df.to_excel('./files/1_crawling_raw.xlsx', index = False)